# ETL Validation Framework: Teradata to Snowflake

**Generated on:** 2025-05-05 04:34:56

## Overview
This notebook demonstrates an ETL validation framework built in Python. It validates data integrity during migration from **Teradata** to **Snowflake**. The checks include:
- Row count validation
- Null check
- Duplicate check


## Step 1: Install Required Packages

In [ ]:
!pip install teradatasql snowflake-connector-python

## Step 2: Load Configurations

In [ ]:
import json

with open('config/connections.json') as f:
    config = json.load(f)

## Step 3: Define Logger

In [ ]:
from datetime import datetime

def log_to_file(message):
    with open("logs/validation.log", "a") as f:
        f.write(f"[{datetime.now()}] {message}\n")

## Step 4: Row Count Validation

In [ ]:
import teradatasql
import snowflake.connector

def validate_row_count(td_conn, sf_conn):
    with td_conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM source.customer")
        td_count = cur.fetchone()[0]
    with sf_conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM customer")
        sf_count = cur.fetchone()[0]
    assert td_count == sf_count, f"Row count mismatch: Teradata({td_count}) vs Snowflake({sf_count})"

## Step 5: Null Check Validation

In [ ]:
def validate_null_check(sf_conn, table, columns):
    with sf_conn.cursor() as cur:
        for col in columns:
            cur.execute(f"SELECT COUNT(*) FROM {table} WHERE {col} IS NULL")
            null_count = cur.fetchone()[0]
            assert null_count == 0, f"Nulls found in column {col}"

## Step 6: Duplicate Check Validation

In [ ]:
def validate_duplicate_check(sf_conn, table, keys):
    key_expr = ", ".join(keys)
    query = f"SELECT {key_expr}, COUNT(*) FROM {table} GROUP BY {key_expr} HAVING COUNT(*) > 1"
    with sf_conn.cursor() as cur:
        cur.execute(query)
        rows = cur.fetchall()
        assert not rows, f"Duplicates found on keys {keys}: {rows}"

## Step 7: Run Validations

In [ ]:
def run_validations():
    td_conn = teradatasql.connect(**config['teradata'])
    sf_conn = snowflake.connector.connect(**config['snowflake'])
    try:
        validate_row_count(td_conn, sf_conn)
        validate_null_check(sf_conn, "customer", ["email"])
        validate_duplicate_check(sf_conn, "customer", ["customer_id"])
        log_to_file("✅ All validations passed")
    except Exception as e:
        log_to_file(f"❌ Validation failed: {str(e)}")
        raise

run_validations()